### fast local attention + hidden states   

#### batch size = 64, learning rate = 0.001, windows size =  5, num_gpus = 6

In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib
from collections import Counter
import time

VERY_BIG_NUMBER = 1e30

In [ ]:
f = open('../../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()

word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

# Model Description
model_name = 'model-aw-lex-local-att-fast-v2-8'
model_dir = '../output/all-word/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

with open('../../../dataset/train_val_data_fine/all_word_lex','rb') as f:
    train_data, val_data = pickle.load(f)    
    
# Parameters
mode = 'train'
num_senses = 45
num_pos = 12
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 256
num_filter = 256
window_size = 5
kernel_size = 5
keep_prob = 0.3
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 500
decay_rate = 0.99
clip_norm = 1
clipping = True
moving_avg_deacy = 0.999
num_gpus = 6
width = int(window_size/2)

In [ ]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, axis=0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [ ]:
# MODEL
device_num = 0
tower_grads = []
losses = []
predictions = []
predictions_pos = []

x = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="y")
y_pos = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="y")
x_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='x_mask') 
sense_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='sense_mask')
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
pretrain = tf.placeholder('bool', [], name="pretrain")

global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)
summaries = []

def global_attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_masked, a), axis=0)  
    return c

with tf.variable_scope("word_embedding"):
    unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)

with tf.variable_scope(tf.get_variable_scope()):
    for gpu_idx in range(num_gpus):
#         if gpu_idx>=4:
#             device_num = 1
#         if gpu_idx>=5:
#             device_num = 2
        with tf.name_scope("model_{}".format(gpu_idx)) as scope, tf.device('/gpu:%d' % device_num):

            if gpu_idx > 0:
                    tf.get_variable_scope().reuse_variables()

            with tf.name_scope("word"):
                Wx = tf.nn.embedding_lookup(final_word_emb_mat, x[gpu_idx])  

            float_x_mask = tf.cast(x_mask[gpu_idx], 'float')
            x_len = tf.reduce_sum(tf.cast(x_mask[gpu_idx], 'int32'), axis=1)
            
            with tf.variable_scope("lstm1"):
                cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
                d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)

                (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
                h1 = tf.concat([fw_h1, bw_h1], 2)

            with tf.variable_scope("lstm2"):
                cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
                d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)

                (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
                h = tf.concat([fw_h2, bw_h2], 2)
            
            with tf.variable_scope("local_attention"):
                W_att_local = tf.get_variable("W_att_local", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*10))
                flat_h = tf.reshape(h, [batch_size*max_sent_size, tf.shape(h)[2]])
                h_tanh = tf.tanh(flat_h)
                u_flat = tf.matmul(h_tanh, W_att_local)
                u_local = tf.reshape(u_flat, [batch_size, max_sent_size])
                final_u = (tf.cast(x_mask[gpu_idx], 'float') -1)*VERY_BIG_NUMBER + u_local
                c_local = tf.map_fn(lambda i:tf.reduce_sum(tf.multiply(h[:, tf.maximum(0, i-width-1):tf.minimum(1+width+i, max_sent_size)],
                    tf.expand_dims(tf.nn.softmax(final_u[:, tf.maximum(0, i-width-1):tf.minimum(1+width+i, max_sent_size)], 1), 2)), axis=1),
                    tf.range(max_sent_size), dtype=tf.float32)  
            
            c_local = tf.transpose(c_local, perm=[1,0,2]) 
            h_final = tf.concat([tf.multiply(c_local, tf.expand_dims(float_x_mask, 2)), h], 2)
            flat_h_final = tf.reshape(h_final, [-1, tf.shape(h_final)[2]])
            with tf.variable_scope("hidden_layer"):
                W = tf.get_variable("W", shape=[4*hidden_size, 2*hidden_size], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*20))
                b = tf.get_variable("b", shape=[2*hidden_size], initializer=tf.zeros_initializer())
                drop_flat_h_final = tf.nn.dropout(flat_h_final, input_keep_prob)
                flat_hl = tf.matmul(drop_flat_h_final, W) + b

            with tf.variable_scope("softmax_layer"):
                W = tf.get_variable("W", shape=[2*hidden_size, num_senses], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*20))
                b = tf.get_variable("b", shape=[num_senses], initializer=tf.zeros_initializer())
                drop_flat_hl = tf.nn.dropout(flat_hl, input_keep_prob)
                flat_logits_sense = tf.matmul(drop_flat_hl, W) + b
                logits = tf.reshape(flat_logits_sense, [batch_size, max_sent_size, num_senses])
                predictions.append(tf.argmax(logits, 2))

            with tf.variable_scope("softmax_layer_pos"):
                W = tf.get_variable("W", shape=[2*hidden_size, num_pos], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*30))
                b = tf.get_variable("b", shape=[num_pos], initializer=tf.zeros_initializer())
                flat_h1 = tf.reshape(h1, [-1, tf.shape(h1)[2]])
                drop_flat_hl = tf.nn.dropout(flat_hl, input_keep_prob)
                flat_logits_pos = tf.matmul(drop_flat_hl, W) + b
                logits_pos = tf.reshape(flat_logits_pos, [batch_size, max_sent_size, num_pos])
                predictions_pos.append(tf.argmax(logits_pos, 2))


            float_sense_mask = tf.cast(sense_mask[gpu_idx], 'float')

            loss = tf.contrib.seq2seq.sequence_loss(logits, y[gpu_idx], float_sense_mask, name="loss")
            loss_pos = tf.contrib.seq2seq.sequence_loss(logits_pos, y_pos[gpu_idx], float_x_mask, name="loss_")

            l2_loss = l2_lambda * tf.losses.get_regularization_loss()

            total_loss = tf.cond(pretrain, lambda:loss_pos, lambda:loss + loss_pos + l2_loss)

            summaries.append(tf.summary.scalar("loss_{}".format(gpu_idx), loss))
            summaries.append(tf.summary.scalar("loss_pos_{}".format(gpu_idx), loss_pos))
            summaries.append(tf.summary.scalar("total_loss_{}".format(gpu_idx), total_loss))


            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_vars = optimizer.compute_gradients(total_loss)

            clipped_grads = grads_vars
            if(clipping == True):
                clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

            tower_grads.append(clipped_grads)
            losses.append(total_loss)

with tf.device('/gpu:0'):
    tower_grads = average_gradients(tower_grads)
    losses = tf.add_n(losses)/len(losses)
    apply_grad_op = optimizer.apply_gradients(tower_grads, global_step=global_step)
    summaries.append(tf.summary.scalar('total_loss', losses))
    summaries.append(tf.summary.scalar('learning_rate', learning_rate))

    variable_averages = tf.train.ExponentialMovingAverage(moving_avg_deacy, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    train_op = tf.group(apply_grad_op, variables_averages_op)
    saver = tf.train.Saver(tf.global_variables())
    summary = tf.summary.merge(summaries)

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# print (device_lib.list_local_devices())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [ ]:
save_period = 100
log_period = 100

def model(xx, yy, yy_pos, mask, smask, train_cond=True, pretrain_cond=False):
    num_batches = int(len(xx)/(batch_size*num_gpus))
    _losses = 0
    temp_loss = 0
    preds_sense = []
    true_sense = []
    preds_pos = []
    true_pos = []
    
    for j in range(num_batches): 
        
        s = j * batch_size * num_gpus
        e = (j+1) * batch_size * num_gpus
        xx_re = xx[s:e].reshape([num_gpus, batch_size, -1])
        yy_re = yy[s:e].reshape([num_gpus, batch_size, -1])
        yy_pos_re = yy_pos[s:e].reshape([num_gpus, batch_size, -1])
        mask_re = mask[s:e].reshape([num_gpus, batch_size, -1])
        smask_re = smask[s:e].reshape([num_gpus, batch_size, -1])
 
        feed_dict = {x:xx_re, y:yy_re, y_pos:yy_pos_re, x_mask:mask_re, sense_mask:smask_re, pretrain:pretrain_cond, is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, losses, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)
            
            temp_loss += _loss
            if((j+1)%log_period==0):
                print("Steps: {}".format(step), "Loss:{0:.4f}".format(temp_loss/log_period), ", Current Loss: {0:.4f}".format(_loss))
                temp_loss = 0
            if((j+1)%save_period==0):
                saver.save(sess, save_path=save_dir)                         
                
        else:
            _loss, pred, pred_pos = sess.run([total_loss, predictions, predictions_pos], feed_dict)
            for i in range(num_gpus):
                preds_sense.append(pred[i][smask_re[i]])
                true_sense.append(yy_re[i][smask_re[i]])
                preds_pos.append(pred_pos[i][mask_re[i]])
                true_pos.append(yy_pos_re[i][mask_re[i]])

        _losses +=_loss

    if(train_cond==False): 
        sense_preds = []
        sense_true = []
        pos_preds = []
        pos_true = []
        
        for preds in preds_sense:
            for ps in preds:      
                sense_preds.append(ps)  
        for trues in true_sense:
            for ts in trues:
                sense_true.append(ts)
        
        for preds in preds_pos:
            for ps in preds:      
                pos_preds.append(ps)      
        for trues in true_pos:
            for ts in trues:
                pos_true.append(ts)
                
        return _losses/num_batches, sense_preds, sense_true, pos_preds, pos_true

    return _losses/num_batches, step

def eval_score(yy, pred, yy_pos, pred_pos):
    f1 = f1_score(yy, pred, average='macro')
    accu = accuracy_score(yy, pred)
    f1_pos = f1_score(yy_pos, pred_pos, average='macro')
    accu_pos = accuracy_score(yy_pos, pred_pos)
    return f1*100, accu*100, f1_pos*100, accu_pos*100

In [ ]:
x_id_train = train_data['x']
mask_train = train_data['x_mask']
sense_mask_train = train_data['sense_mask']
y_train = train_data['y']
y_pos_train = train_data['pos']

x_id_val = val_data['x']
mask_val = val_data['x_mask']
sense_mask_val = val_data['sense_mask']
y_val = val_data['y']
y_pos_val = val_data['pos']

In [ ]:
def testing():
    start_time = time.time()
    val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
    f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
    time_taken = time.time() - start_time
    print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))
    return f1_, accu_, f1_pos_, accu_pos_

def training(current_epoch, pre_train_cond):
        random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
        x_id_train_tmp = x_id_train[random]
        y_train_tmp = y_train[random]
        mask_train_tmp = mask_train[random]    
        sense_mask_train_tmp = sense_mask_train[random]
        y_pos_train_tmp = y_pos_train[random]

        start_time = time.time()
        train_loss, step = model(x_id_train_tmp, y_train_tmp, y_pos_train_tmp, mask_train_tmp, sense_mask_train_tmp, pretrain_cond=pre_train_cond)
        time_taken = time.time() - start_time
        print("Epoch: {}".format(current_epoch+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        return [step, train_loss]

In [ ]:
loss_collection = []
val_collection = []
num_epochs = 20
val_period = 2

# Pretraining POS Tags
training(0, True)
training(1, True)
testing()

for i in range(num_epochs):
    loss_collection.append(training(i, False))
    if((i+1)%val_period==0):
        val_collection.append(testing())

Steps: 100 Loss:1.6658 , Current Loss: 0.8500
Steps: 200 Loss:0.6862 , Current Loss: 0.5828
Steps: 300 Loss:0.5355 , Current Loss: 0.5045
Steps: 400 Loss:0.4562 , Current Loss: 0.4392
Steps: 500 Loss:0.4081 , Current Loss: 0.3960
Steps: 600 Loss:0.3770 , Current Loss: 0.3757
Steps: 700 Loss:0.3519 , Current Loss: 0.3404
Steps: 800 Loss:0.3346 , Current Loss: 0.3362
Steps: 900 Loss:0.3201 , Current Loss: 0.3170
Steps: 1000 Loss:0.3086 , Current Loss: 0.2949
Steps: 1100 Loss:0.2965 , Current Loss: 0.2946
Steps: 1200 Loss:0.2895 , Current Loss: 0.2857
Steps: 1300 Loss:0.2808 , Current Loss: 0.2706
Steps: 1400 Loss:0.2734 , Current Loss: 0.2664
Steps: 1500 Loss:0.2677 , Current Loss: 0.2717
Steps: 1600 Loss:0.2605 , Current Loss: 0.2667
Steps: 1700 Loss:0.2560 , Current Loss: 0.2493
Epoch: 1 , Step: 1771 , loss: 0.4262 , Time: 12779.7
Model Saved
Steps: 1871 Loss:0.2488 , Current Loss: 0.2442
Steps: 1971 Loss:0.2436 , Current Loss: 0.2389
Steps: 2071 Loss:0.2394 , Current Loss: 0.2319
Step

/users/btech/aviraj/envs/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Val: F1 Score:1.33 Accuracy:2.10  POS: F1 Score:88.60 Accuracy:93.15 Loss:8.1548 , Time: 2248.4
Steps: 3642 Loss:3.0114 , Current Loss: 2.6059
Steps: 3742 Loss:2.6016 , Current Loss: 2.5623
Steps: 3842 Loss:2.4766 , Current Loss: 2.3481
Steps: 3942 Loss:2.4218 , Current Loss: 2.4654
Steps: 4042 Loss:2.3920 , Current Loss: 2.3134
Steps: 4142 Loss:2.3634 , Current Loss: 2.3470
Steps: 4242 Loss:2.3332 , Current Loss: 2.1992
Steps: 4342 Loss:2.3062 , Current Loss: 2.2840
Steps: 4442 Loss:2.2944 , Current Loss: 2.3044
Steps: 4542 Loss:2.2695 , Current Loss: 2.2519
Steps: 4642 Loss:2.2503 , Current Loss: 2.2373
Steps: 4742 Loss:2.2353 , Current Loss: 2.2210
Steps: 4842 Loss:2.2189 , Current Loss: 2.2031
Steps: 4942 Loss:2.2078 , Current Loss: 2.2479
Steps: 5042 Loss:2.1870 , Current Loss: 2.1339
Steps: 5142 Loss:2.1976 , Current Loss: 2.1943
Steps: 5242 Loss:2.1772 , Current Loss: 2.0594
Epoch: 1 , Step: 5313 , loss: 2.3419 , Time: 12284.0
Model Saved
Steps: 5413 Loss:2.1516 , Current Loss: 

In [ ]:
loss_collection = []
val_collection = []
num_epochs = 20
val_period = 2

for i in range(num_epochs):
    loss_collection.append(training(i, False))
    if((i+1)%val_period==0):
        val_collection.append(testing())

Steps: 7313 Loss:2.0441 , Current Loss: 2.0908
Steps: 7413 Loss:2.0285 , Current Loss: 1.9964
Steps: 7513 Loss:2.0299 , Current Loss: 1.8709
Steps: 7613 Loss:2.0208 , Current Loss: 2.0390
Steps: 7713 Loss:2.0292 , Current Loss: 2.0561
Steps: 7813 Loss:2.0199 , Current Loss: 2.1160
Steps: 7913 Loss:2.0161 , Current Loss: 2.1142
Steps: 8013 Loss:2.0166 , Current Loss: 1.8492
Steps: 8113 Loss:2.0053 , Current Loss: 1.9138
Steps: 8213 Loss:2.0112 , Current Loss: 1.9912
Steps: 8313 Loss:2.0091 , Current Loss: 1.9781
Steps: 8413 Loss:1.9832 , Current Loss: 1.9294
Steps: 8513 Loss:1.9915 , Current Loss: 1.9281
Steps: 8613 Loss:1.9936 , Current Loss: 1.9739
Steps: 8713 Loss:1.9923 , Current Loss: 1.9497
Steps: 8813 Loss:1.9851 , Current Loss: 2.0962
Steps: 8913 Loss:1.9810 , Current Loss: 2.0121
Epoch: 1 , Step: 8984 , loss: 2.0082 , Time: 14062.4
Model Saved
Steps: 9084 Loss:1.9731 , Current Loss: 1.9911
Steps: 9184 Loss:1.9717 , Current Loss: 2.0535
Steps: 9284 Loss:1.9665 , Current Loss: 1.

In [ ]:
loss_collection = []
val_collection = []
num_epochs = 20
val_period = 2

for i in range(num_epochs):
    loss_collection.append(training(i, False))
    if((i+1)%val_period==0):
        val_collection.append(testing())

Steps: 9784 Loss:1.9669 , Current Loss: 1.9211
Steps: 9884 Loss:1.9610 , Current Loss: 1.8971
Steps: 9984 Loss:1.9563 , Current Loss: 1.9640
Steps: 10084 Loss:1.9494 , Current Loss: 1.8538
Steps: 10184 Loss:1.9473 , Current Loss: 1.8563
Steps: 10284 Loss:1.9419 , Current Loss: 1.8650
Steps: 10384 Loss:1.9399 , Current Loss: 1.9312
Steps: 10484 Loss:1.9433 , Current Loss: 1.8442
Steps: 10584 Loss:1.9217 , Current Loss: 1.8807
Steps: 10684 Loss:1.9199 , Current Loss: 1.9798
Steps: 10784 Loss:1.9399 , Current Loss: 1.9384
Steps: 10884 Loss:1.9311 , Current Loss: 1.9601
Steps: 10984 Loss:1.9350 , Current Loss: 1.7945
Steps: 11084 Loss:1.9105 , Current Loss: 1.9698
Steps: 11184 Loss:1.9217 , Current Loss: 1.9404
Steps: 11284 Loss:1.9256 , Current Loss: 1.9251
Steps: 11384 Loss:1.9123 , Current Loss: 1.9369
Epoch: 1 , Step: 11455 , loss: 1.9354 , Time: 16339.0
Model Saved
Steps: 11555 Loss:1.8978 , Current Loss: 1.8777
Steps: 11655 Loss:1.9140 , Current Loss: 1.9409
Steps: 11755 Loss:1.9068 

In [ ]:
loss_collection = []
val_collection = []
num_epochs = 20
val_period = 2

for i in range(num_epochs):
    loss_collection.append(training(i, False))
    if((i+1)%val_period==0):
        val_collection.append(testing())

Steps: 12255 Loss:1.8887 , Current Loss: 1.8646
Steps: 12355 Loss:1.8861 , Current Loss: 1.9412
Steps: 12455 Loss:1.9008 , Current Loss: 1.9083
Steps: 12555 Loss:1.8908 , Current Loss: 1.9083
Steps: 12655 Loss:1.8892 , Current Loss: 1.9260
Steps: 12755 Loss:1.8921 , Current Loss: 1.8409
Steps: 12855 Loss:1.8859 , Current Loss: 1.8544
Steps: 12955 Loss:1.8736 , Current Loss: 2.0098
Steps: 13055 Loss:1.8837 , Current Loss: 1.8860
Steps: 13155 Loss:1.8861 , Current Loss: 1.8802
Steps: 13255 Loss:1.8705 , Current Loss: 1.9300
Steps: 13355 Loss:1.8874 , Current Loss: 1.8558
Steps: 13455 Loss:1.8831 , Current Loss: 1.9937
Steps: 13555 Loss:1.8668 , Current Loss: 1.7420
Steps: 13655 Loss:1.8646 , Current Loss: 1.9197
Steps: 13755 Loss:1.8651 , Current Loss: 1.8406
Steps: 13855 Loss:1.8712 , Current Loss: 1.8842
Epoch: 1 , Step: 13926 , loss: 1.8803 , Time: 14173.8
Model Saved
Steps: 14026 Loss:1.8535 , Current Loss: 1.7385
Steps: 14126 Loss:1.8702 , Current Loss: 1.8798
Steps: 14226 Loss:1.86

In [ ]:
start_time = time.time()
train_loss, train_pred, train_true, train_pred_pos, train_true_pos = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, train_cond=False)        
f1_, accu_, f1_pos_, accu_pos_ = etrain_score(train_true, train_pred, train_true_pos, train_pred_pos)
time_taken = time.time() - start_time
print("train: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))

In [9]:
saver.restore(sess, save_dir)